## Build a Wflow model from scratch

This notebook demonstrates how to prepare **Wflow** model from scratch using the command line interace (CLI).

All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### HydroMT CLI build interface

Lets first check if the Wflow model is recognized by HydroMT

# SELECT CORRECT Kernal

In [1]:
import hydromt
!hydromt --models

Model plugins:
	- model (hydromt 1.3.0rc5)
	- example_model (hydromt 1.3.0rc5)
	- wflow_sbm (hydromt_wflow 1.0.0.dev0)
	- wflow_sediment (hydromt_wflow 1.0.0.dev0)


In [24]:
!hydromt --models

Model plugins:
	- model (hydromt 1.3.0rc5)
	- example_model (hydromt 1.3.0rc5)
	- wflow_sbm (hydromt_wflow 1.0.0.dev0)
	- wflow_sediment (hydromt_wflow 1.0.0.dev0)


In [29]:
!hydromt build --help

Usage: hydromt build [OPTIONS] MODEL MODEL_ROOT

  Build models from scratch.

  Example usage: --------------

  To build a wflow model: hydromt build wflow_sbm /path/to/model_root -i
  /path/to/wflow_config.yml -d deltares_data -d /path/to/data_catalog.yml -v

  To build a sfincs model: hydromt build sfincs /path/to/model_root  -i
  /path/to/sfincs_config.yml -d /path/to/data_catalog.yml -v

Options:
  -i, --config PATH        Path to hydroMT configuration file, for the model
                           specific implementation.  [required]
  -d, --data TEXT          Path to local yaml data catalog file OR name of
                           predefined data catalog.
  --dd, --deltares-data    Flag: Shortcut to add the "deltares_data" catalog
  --fo, --force-overwrite  Flag: If provided overwrite existing model files
  --cache                  Flag: If provided cache tiled rasterdatasets
  -v, --verbose            Increase verbosity.
  -q, --quiet              Decrease verbosity.
  --hel

In [ ]:
-r "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}"

In [30]:
!hydromt build wflow_sbm "./wflow_test_full"  -i wflow_build.yml -d artifact_data -vv

2025-11-18 15:22:30,122 - hydromt - log - INFO - HydroMT version: 1.3.0-rc5
2025-11-18 15:22:30,128 - hydromt.readers - readers - ERROR - It seems your workflow file at /home/test2/Capacity-Building-ICPAC-for-climate-services/2_IBF_Flood_Damage/wflow_build.yml does not contain a `steps` section. Perhaps you're using a v0.x format? 
Traceback (most recent call last):
  File "/opt/icpac-course/hydromt_wflow/.pixi/envs/default/bin/hydromt", line 10, in <module>
    sys.exit(main())
             ~~~~^^
  File "/opt/icpac-course/hydromt_wflow/.pixi/envs/default/lib/python3.13/site-packages/click/core.py", line 1462, in __call__
    return self.main(*args, **kwargs)
           ~~~~~~~~~^^^^^^^^^^^^^^^^^
  File "/opt/icpac-course/hydromt_wflow/.pixi/envs/default/lib/python3.13/site-packages/click/core.py", line 1383, in main
    rv = self.invoke(ctx)
  File "/opt/icpac-course/hydromt_wflow/.pixi/envs/default/lib/python3.13/site-packages/click/core.py", line 1850, in invoke
    return _process

In [7]:
# this should return "wflow, wflow_sediment"
# as well as the generic HydroMT models "grid_model, lumped_model, network_model, (mesh_model)"
!hydromt --models

Model plugins:
	- model (hydromt 1.2.0.dev0)
	- example_model (hydromt 1.2.0.dev0)


Using the **HydroMT build** API we can setup a complete model from scratch. Let's get an overview of all the available options:

### Model setup configuration

The HydroMT configuration file (YAML) contains the model setup configuration and determines which methods are used to prepare the different components of a Wflow model and in which order and optionally sets non-default arguments for each method. This configuration is passed to HydroMT using `-i <path_to_config_file>`. We have prepared several example files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_wflow/tree/main/examples) and from the [docs (building a model)](https://deltares.github.io/hydromt_wflow/latest/user_guide/wflow_build). 

Each header (without indent) (e.g. `setup_basemaps:`) corresponds to a model method. All model methods are explained in the [docs (model components)](https://deltares.github.io/hydromt_wflow/latest/user_guide/wflow_model_setup.html). 

We will load the default wflow build configuration yaml file for inspection:

In [13]:
fn_config = "/data/shared/hydroMT_data/hydromt_dire_dawa/local_data.yml"

with open(fn_config, "r") as f:
    txt = f.read()
print(txt)

esa_worldcover_2021:
  crs: 4326
  data_type: RasterDataset
  driver: raster
  kwargs:
    chunks:
      x: 3600
      y: 3600
  meta:
    category: landuse
  path: esa_worldcover_2021.tif
Delta_DTM:
  crs: 4326
  data_type: RasterDataset
  driver: raster
  kwargs:
    chunks:
      x: 3600
      y: 3600
  meta:
    category: elevation
  path: delta_DTM.tif
soilgrids_2020:
  alias: soilgrids_v2.0
soilgrids_v2.0:
  data_type: RasterDataset
  driver: raster
  kwargs:
    chunks:
      x: 2400
      y: 2400
  meta:
    category: soil
    notes: "soilthickness is based on 1) soilgrids (global) and 2) dataset for Eurasia\
      \ (ESDAC, 2004; Panagos et al., 2012): ESDAC, 2004. The european soil database\
      \ distribution version 2.0, european commission and the European soil bureau\
      \ network.  esdac.jrc.ec.europa.eu, accessed: 2017-11-17.  Panagos, P., Van\
      \ Liedekerke, M., Jones, A., Montanarella, L., 2012. European soil data centre:\
      \ Response to european policy

### Data for model setup

The `<...>_fn` arguments correspond to a data source from the `DataCatalog` which is based on a yml-file with references to the data paths/urls and how the data should be read. This file can be provided to hydromt using `-d <path_to_yml_file>`. 

By default, the pre-defined `artifact_data` catalog containing some example data for the Piave basin will be downloaded to `~/.hydromt_data/` which is also used for this example. An overview of the available example data is provided [here](https://deltares.github.io/hydromt/latest/user_guide/data_existing_cat.html)
This example data is a based on the data which is available from the Deltares p-drive. 
If you have acces to this drive, a pre-configured catalog file can be loaded using the `-d deltares_data` pre-defined catalog. 

More background how to write a data catalog yml file can be found in the [HydroMT core docs](https://deltares.github.io/hydromt/latest/user_guide/data_prepare_cat.html) 

### Setup complete Wflow model 

Once we have prepared our data catalog and model build settings file, we are ready to ask HydroMT to build our Wflow model. Building the model can be done from the command line. Down below is an example for the Northern part of the Piave basin in Italy:

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build wflow "./wflow_model_output" -r "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}" -i wflow_build.yml -d artifact_data -vv

In [26]:
!hydromt build wflow_sbm "./wflow_model_output" -r "{'subbasin': [42.02, 9.1200], 'strord': 4}" -i "/data/shared/hydroMT_data/hydromt_dire_dawa/wflow_build_dire_dawa.yml" -d "/data/shared/hydroMT_data/hydromt_dire_dawa/local_data.yml" -vv

Usage: hydromt build [OPTIONS] MODEL MODEL_ROOT
Try 'hydromt build --help' for help.

Error: No such option: -r


With this example we build a complete Wflow model including forcing. The example means the following: run **hydromt build** with:

* `wflow` : i.e. build a Wflow model
* `./wflow_test_full` : output model folder
* `-r "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}"` : derive a subbasin with its outlet at the given x,y coordinates and within the given bounds [xmin, ymin, xmax, ymax] (WGS84) snapped to a river with minimum stream order (strord) of 4. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/model_region)
* `-i wflow_build.yml` : setup configuration file including all components to build and their arguments
* `-d artifact_data`: data catalog to use. Here `artifact_data` which contains a global data extract for Northern Italy for demo purposes.
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.


Next we check which files have been created. The model root should contain two netcdf (.nc) files, one for the staticmaps and one for the forcing data, a wflow run configuration (.toml) file. These files are sufficient to run the wflow model. In addition several geometry (.geojson) files are saved in the *staticgeoms* folder.  Finally, the setup process is logged in the hydromt.log file. 

In [ ]:
import os

root = "wflow_test_full"
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        if name.endswith(".xml"):
            continue
        print(f" - {name}")

### Visualize and/or inspect model schematization

* The **wflow plot** example notebook contains scripts to visualize your model
* The **wflow nc to raster files** example notebook contains scripts to write the nc model files to a netcdf which can be used to inspect (and modify) the model layers in e.g. QGIS.